# IMDB 영화리뷰 감성분석 - Conv1D
- Conv1D + Conv1D + Dense
- Conv1D + LSTM + Dense
- Conv1D + Dense + Dense

In [1]:
import numpy as np
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [3]:
num_words = 10000
(X_train, y_train), (_, _) = imdb.load_data(num_words=num_words)

17473536/17464789 [==============================] - 0s 0us/step


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train, stratify=y_train, test_size=0.2, random_state=seed
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20000,), (5000,), (20000,), (5000,))

In [5]:
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_train.shape, X_test.shape

((20000, 500), (5000, 500))

- Case 1) Conv1D x 2

In [6]:
model1 = Sequential([ 
    Embedding(num_words, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 7, activation='relu'),
    MaxPooling1D(7),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(5),
    GlobalMaxPooling1D(),
    Dense(1, activation='sigmoid')
])
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 100)          1000000   
                                                                 
 dropout (Dropout)           (None, 500, 100)          0         
                                                                 
 conv1d (Conv1D)             (None, 494, 64)           44864     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 70, 64)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 66, 64)            20544     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 13, 64)           0         
 1D)                                                    

In [7]:
model1.compile('adam', 'binary_crossentropy', ['accuracy'])
model1_path = 'best-imdb-conv1d-conv1d.h5'
mc1 = ModelCheckpoint(model1_path, verbose=1, save_best_only=True)
es1 = EarlyStopping(patience=5)

In [8]:
hist1 = model1.fit(
    X_train, y_train, validation_split=0.2,
    epochs=30, batch_size=64, callbacks=[mc1,es1]
)

Epoch 1/30
247/250 [============================>.] - ETA: 0s - loss: 0.5592 - accuracy: 0.6766
Epoch 1: val_loss improved from inf to 0.36014, saving model to best-imdb-conv1d-conv1d.h5
250/250 [==============================] - 15s 10ms/step - loss: 0.5563 - accuracy: 0.6791 - val_loss: 0.3601 - val_accuracy: 0.8475
Epoch 2/30
250/250 [==============================] - ETA: 0s - loss: 0.2874 - accuracy: 0.8832
Epoch 2: val_loss improved from 0.36014 to 0.32265, saving model to best-imdb-conv1d-conv1d.h5
250/250 [==============================] - 2s 9ms/step - loss: 0.2874 - accuracy: 0.8832 - val_loss: 0.3227 - val_accuracy: 0.8635
Epoch 3/30
248/250 [============================>.] - ETA: 0s - loss: 0.1945 - accuracy: 0.9262
Epoch 3: val_loss did not improve from 0.32265
250/250 [==============================] - 2s 9ms/step - loss: 0.1944 - accuracy: 0.9261 - val_loss: 0.3540 - val_accuracy: 0.8540
Epoch 4/30
248/250 [============================>.] - ETA: 0s - loss: 0.1322 - accur

In [9]:
best_model1 = load_model(model1_path)
best_model1.evaluate(X_test, y_test)

157/157 [==============================] - 1s 4ms/step - loss: 0.3083 - accuracy: 0.8684


[0.3083183467388153, 0.868399977684021]

- Case 2) Conv1D + LSTM

In [10]:
model2 = Sequential([ 
    Embedding(num_words, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(5),
    LSTM(100),
    Dense(1, activation='sigmoid')
])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_1 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 496, 64)           32064     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 99, 64)           0         
 1D)                                                             
                                                                 
 lstm (LSTM)                 (None, 100)               66000     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                      

In [11]:
model2.compile('adam', 'binary_crossentropy', ['accuracy'])
model2_path = 'best-imdb-conv1d-lstm.h5'
mc2 = ModelCheckpoint(model2_path, verbose=1, save_best_only=True)
es2 = EarlyStopping(patience=5)

In [12]:
hist2 = model2.fit(
    X_train, y_train, validation_split=0.2,
    epochs=30, batch_size=64, callbacks=[mc2,es2]
)

Epoch 1/30
247/250 [============================>.] - ETA: 0s - loss: 0.4429 - accuracy: 0.7658
Epoch 1: val_loss improved from inf to 0.29654, saving model to best-imdb-conv1d-lstm.h5
250/250 [==============================] - 6s 16ms/step - loss: 0.4410 - accuracy: 0.7674 - val_loss: 0.2965 - val_accuracy: 0.8815
Epoch 2/30
247/250 [============================>.] - ETA: 0s - loss: 0.2173 - accuracy: 0.9150
Epoch 2: val_loss improved from 0.29654 to 0.26502, saving model to best-imdb-conv1d-lstm.h5
250/250 [==============================] - 4s 14ms/step - loss: 0.2180 - accuracy: 0.9148 - val_loss: 0.2650 - val_accuracy: 0.8905
Epoch 3/30
248/250 [============================>.] - ETA: 0s - loss: 0.1426 - accuracy: 0.9490
Epoch 3: val_loss did not improve from 0.26502
250/250 [==============================] - 4s 14ms/step - loss: 0.1426 - accuracy: 0.9489 - val_loss: 0.2914 - val_accuracy: 0.8823
Epoch 4/30
247/250 [============================>.] - ETA: 0s - loss: 0.0969 - accuracy

In [13]:
best_model2 = load_model(model2_path)
best_model2.evaluate(X_test, y_test)

157/157 [==============================] - 1s 5ms/step - loss: 0.2720 - accuracy: 0.8888


[0.27202364802360535, 0.8888000249862671]

- Case 3) Conv1D + Dense

In [14]:
model3 = Sequential([ 
    Embedding(num_words, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(5),
    GlobalMaxPooling1D(),
    Dense(100, activation='relu'),
    Dense(1, activation='sigmoid')
])
model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_2 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_3 (Conv1D)           (None, 496, 64)           32064     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 99, 64)           0         
 1D)                                                             
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_2 (Dense)             (None, 100)              

In [15]:
model3.compile('adam', 'binary_crossentropy', ['accuracy'])
model3_path = 'best-imdb-conv1d-dense.h5'
mc3 = ModelCheckpoint(model3_path, verbose=1, save_best_only=True)
es3 = EarlyStopping(patience=5)

In [16]:
hist3 = model3.fit(
    X_train, y_train, validation_split=0.2,
    epochs=30, batch_size=64, callbacks=[mc3,es3]
)

Epoch 1/30
245/250 [============================>.] - ETA: 0s - loss: 0.5826 - accuracy: 0.6620
Epoch 1: val_loss improved from inf to 0.38888, saving model to best-imdb-conv1d-dense.h5
250/250 [==============================] - 3s 9ms/step - loss: 0.5789 - accuracy: 0.6652 - val_loss: 0.3889 - val_accuracy: 0.8282
Epoch 2/30
244/250 [============================>.] - ETA: 0s - loss: 0.3134 - accuracy: 0.8688
Epoch 2: val_loss improved from 0.38888 to 0.30136, saving model to best-imdb-conv1d-dense.h5
250/250 [==============================] - 2s 9ms/step - loss: 0.3144 - accuracy: 0.8685 - val_loss: 0.3014 - val_accuracy: 0.8742
Epoch 3/30
244/250 [============================>.] - ETA: 0s - loss: 0.2103 - accuracy: 0.9182
Epoch 3: val_loss improved from 0.30136 to 0.29671, saving model to best-imdb-conv1d-dense.h5
250/250 [==============================] - 2s 8ms/step - loss: 0.2099 - accuracy: 0.9185 - val_loss: 0.2967 - val_accuracy: 0.8770
Epoch 4/30
248/250 [=====================

In [17]:
best_model3 = load_model(model3_path)
best_model3.evaluate(X_test, y_test)

157/157 [==============================] - 1s 3ms/step - loss: 0.3079 - accuracy: 0.8740


[0.30789482593536377, 0.8740000128746033]